In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1,3,4,5,6,7"

os.environ['CUDA_PATH']='/usr/local/cuda-11'

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
import re

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
tokenizer.add_special_tokens({'pad_token': '<?>'})

0

In [5]:
from accelerate import init_empty_weights, infer_auto_device_map, load_checkpoint_and_dispatch
from transformers import AutoConfig, AutoModelForCausalLM

In [6]:
from tune_gorilla import prep_model

In [7]:
model = prep_model("/mnt/data/mart/test_trainer/checkpoint-630/")
# model = get_model("/mnt/data/mart/falcon-7b-sharded-bf16/")

In [8]:
model = model.eval()

In [9]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusFor

In [10]:
sequences = pipeline(
   "The user is interested in a tool to find relationships between medical terms.",
    max_length=640,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: The user is interested in a tool to find relationships between medical terms. Create a model to analyze medical texts.
<IDE-genie>: <<<domain>>>: Natural Language Processing Token Classification
<<<api_call>>>: AutoModelForTokenClassification.from_pretrained('d4data/biomedical-ner-all')
<<<api_provider>>>: Transformers
<<<explanation>>>: 1. Import the necessary libraries: AutoTokenizer, AutoModelForTokenClassification, and pipeline from the transformers package.
2. Load the pretrained model 'd4data/biomedical-ner-all' using AutoModelForTokenClassification.from_pretrained(). This model is designed for Named Entity Recognition in the biomedical domain and has been trained on biomedical text data.
3. Load the corresponding tokenizer using AutoTokenizer.from_pretrained() method with the same model name.
4. Create a pipeline object for Named Entity Recognition (NER) using the model and tokenizer.
5. Pass the input text to the NER pipeline to analyze the medical terms and identify th